# Install Modules

# Imports

In [1]:
# 1. Basic Python Imports
import pandas as pd
import numpy as np
import pickle as pi
import os

# 2. API PULL
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# 3. API Keys
from dotenv import load_dotenv


In [2]:
import coinmarketcapapi

cmc = coinmarketcapapi.CoinMarketCapAPI('3486d5cf-6e5b-443f-8f9a-140421f63bb3')

In [3]:
#data = cmc.cryptocurrency_info(symbol='BTC')

In [4]:
#data_id_map = cmc.cryptocurrency_map()

In [5]:
# import pandas as pd
# pd = pd.DataFrame(data_id_map.data, columns =['symbol','price','first_historical_data','market_cap','is_active'])
# pd.set_index('symbol',inplace=True)
# print(pd)

In [6]:
#data_quote = cmc.cryptocurrency_quotes_latest(symbol='ETH', convert='USD')


In [3]:
# api pull
data_listing = cmc.cryptocurrency_listings_latest(convert='USD')

In [4]:
# create df and narrow columns
df = pd.DataFrame.from_dict(data_listing.data)

df = df[['name', 'symbol', 'date_added', 'quote']]
df

,name,symbol,date_added,quote
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,"{'USD': {'price': 36419.12928128711, 'volume_2..."
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,"{'USD': {'price': 2741.097801875638, 'volume_2..."
2,Tether,USDT,2015-02-25T00:00:00.000Z,"{'USD': {'price': 1.0000658202542936, 'volume_..."
3,BNB,BNB,2017-07-25T00:00:00.000Z,"{'USD': {'price': 377.72348771769043, 'volume_..."
4,USD Coin,USDC,2018-10-08T00:00:00.000Z,"{'USD': {'price': 0.9997712619830748, 'volume_..."
...,...,...,...,...
95,yearn.finance,YFI,2020-07-18T00:00:00.000Z,"{'USD': {'price': 16836.0225300365, 'volume_24..."
96,0x,ZRX,2017-08-16T00:00:00.000Z,"{'USD': {'price': 0.7126873917731122, 'volume_..."
97,Moonbeam,GLMR,2022-01-11T12:55:50.000Z,"{'USD': {'price': 2.4331596240005857, 'volume_..."
98,ICON,ICX,2017-10-27T00:00:00.000Z,"{'USD': {'price': 0.6422441111175834, 'volume_..."


In [5]:
# isolate quotes column for expansion into columns
quotes_df = df[['name', 'quote']].copy()
df = df.drop(columns = ['quote'])

In [6]:
# expand into columns
usd = quotes_df["quote"].apply(pd.Series)
price = usd['USD'].apply(pd.Series)

In [8]:
# concat for final df
crypto_df = pd.concat([df, price], axis = 1, join = 'inner')
crypto_df.head(10)

,name,symbol,date_added,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,36419.129281,4.237786e+10,16.5415,-0.288854,-8.384171,-8.434075,-20.752209,-6.969844,-10.079384,6.931191e+11,41.4046,7.648017e+11,2022-05-05T22:23:00.000Z
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,2741.097802,2.241314e+10,24.9483,-0.379081,-6.573604,-6.324847,-20.660376,3.854328,-7.122373,3.307432e+11,19.7575,3.307432e+11,2022-05-05T22:22:00.000Z
2,Tether,USDT,2015-02-25T00:00:00.000Z,1.000066,8.468270e+10,19.0977,0.003093,0.000423,-0.010957,-0.016319,-0.029997,-0.032289,8.315785e+10,4.9676,8.571959e+10,2022-05-05T22:22:00.000Z
3,BNB,BNB,2017-07-25T00:00:00.000Z,377.723488,2.364075e+09,38.3579,-0.104228,-6.034662,-6.880425,-16.127397,-1.139112,-4.398740,6.167355e+10,3.6842,6.236848e+10,2022-05-05T22:22:00.000Z
4,USD Coin,USDC,2018-10-08T00:00:00.000Z,0.999771,5.161968e+09,17.4314,-0.074746,-0.032473,-0.013497,-0.037394,0.021392,-0.049669,4.883843e+10,2.9174,4.883843e+10,2022-05-05T22:22:00.000Z
5,XRP,XRP,2013-08-04T00:00:00.000Z,0.596606,2.147676e+09,10.3964,-0.077090,-7.727456,-7.193739,-27.497238,-19.961019,-6.464483,2.884177e+10,1.7229,5.966058e+10,2022-05-05T22:23:00.000Z
6,Terra,LUNA,2019-07-26T00:00:00.000Z,82.841909,2.255212e+09,17.5065,0.973316,-3.872699,-6.733556,-29.037673,0.865216,60.135607,2.844004e+10,1.6989,6.015493e+10,2022-05-05T22:23:00.000Z
7,Solana,SOL,2020-04-10T00:00:00.000Z,84.562207,1.790329e+09,28.3587,-0.342966,-8.271046,-13.509202,-34.955333,-1.677126,-23.324729,2.827332e+10,1.6899,4.326346e+10,2022-05-05T22:22:00.000Z
8,Cardano,ADA,2017-10-01T00:00:00.000Z,0.790790,1.796010e+09,21.6999,-1.350750,-11.576889,-6.351753,-33.422499,-6.393825,-28.751672,2.669120e+10,1.5954,3.558556e+10,2022-05-05T22:22:00.000Z
9,TerraUSD,UST,2020-09-21T00:00:00.000Z,1.000662,7.316337e+08,22.3445,0.025069,0.037622,0.034832,0.026087,-0.371696,-0.095969,1.868621e+10,1.1158,1.868621e+10,2022-05-05T22:22:00.000Z
